In [2]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

<div class="alert alert-block alert-info">
    
### Question:

What does the programmer affect?
    
</div>

In [5]:
render_code("./programmer/perfstats.c", lang="c++", show="change_cpufrequnecy")

// ./programmer/perfstats.c:71-78 (8 lines)
void change_cpufrequnecy(int MHz) {
    int KHz = MHz*1000;
    char cmdline[1024];
    cpu = sched_getcpu();
    sprintf(cmdline,"/usr/sbin/changefreq %d %d %d",cpu,KHz,KHz);
    system(cmdline);
    return;
}

In [1]:
!cse142 run "cpupower frequency-info -n"

analyzing CPU 0:
  driver: intel_pstate
  CPUs which run at the same hardware frequency: 0
  CPUs which need to have their frequency coordinated by software: 0
  maximum transition latency:  Cannot determine or is not supported.
  hardware limits: 1000.000 MHz - 3.700000 GHz
  available cpufreq governors: performance powersave
  current policy: frequency should be within 1000.000 MHz and 3.700000 GHz.
                  The governor "performance" may decide which speed to use
                  within this range.
  current CPU frequency: Unable to call hardware
  current CPU frequency: 1.000584 GHz (asserted by call to kernel)
  boost state support:
    Supported: yes
    Active: yes


In [3]:
! cse142 run "cat /proc/cpuinfo|grep MHz"

cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000
cpu MHz		: 2300.000


In [4]:
! cse142 run "make -C /nfshome/htseng/courses/CSE142/demo/performance/programmer madd_change_freq"
! cse142 run "perf stat ./programmer/madd_change_freq 8192 1800"
! cse142 run "perf stat ./programmer/madd_change_freq 8192 3700"

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/performance/programmer'
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DA madd_change_freq.c perfstats.o -o madd_change_freq
madd_change_freq.c: In function ‘main’:
madd_change_freq.c:18:3: warning: implicit declaration of function ‘change_cpufrequnecy’ [-Wimplicit-function-declaration]
   18 |   change_cpufrequnecy(freq);
      |   ^~~~~~~~~~~~~~~~~~~
madd_change_freq.c:49:3: warning: implicit declaration of function ‘restore_cpufrequnecy’ [-Wimplicit-function-declaration]
   49 |   restore_cpufrequnecy();
      |   ^~~~~~~~~~~~~~~~~~~~
make: Leaving directory '/nfshome/htseng/courses/CSE142/demo/performance/programmer'
161

 Performance counter stats for './programmer/madd_change_freq 8192 1800':

          3,067.10 msec task-clock                #    0.996 CPUs utilized          
                15      context-switches          #    4.891 /sec                   
                 0      cpu-migrations            #    0.000 /sec    

## Why're we lower the frequency?

In [5]:
! cse142 run "sudo likwid-perfctr -g ENERGY ./programmer/madd_change_freq 8192 1000" | grep STAT

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 10897132232 |   0 | 10508327261 | 1.513491e+08 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  |  4268225703 |   0 |  4073850172 | 5.928091e+07 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  |  9809136968 |   0 |  9369322780 | 1.362380e+08 |
|       TEMP_CORE STAT       |   TMP0  |        2066 |  23 |          34 |      28.6944 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    309.4913 |   0 |    157.2692 |       4.2985 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     66.2007 |   0 |     40.7083 |       0.9195 |
|  Runtime (RDTSC) [s] STAT |   379.1376 |   5.2658 |    5.2658 |   5.2658 |
| Runtime unhalted [s] STAT |     1.8557 |        0 |    1.7713 |   0.0258 |
|      Clock [MHz] STAT     | 38161.8527 | 999.1752 | 2321.5742 | 530.0257 |

In [6]:
! cse142 run "sudo likwid-perfctr -g ENERGY ./programmer/madd_change_freq 8192 2300" | grep STAT

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 10566544874 |   0 | 10508326675 | 1.467576e+08 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  |  4154003399 |   0 |  4115291168 | 5.769449e+07 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  |  4152714216 |   0 |  4115206276 | 5.767659e+07 |
|       TEMP_CORE STAT       |   TMP0  |        2149 |  24 |          36 |      29.8472 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    224.0092 |   0 |    117.4744 |       3.1112 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     30.0867 |   0 |     18.6907 |       0.4179 |
|  Runtime (RDTSC) [s] STAT |   171.2304 |   2.3782 |    2.3782 |   2.3782 |
| Runtime unhalted [s] STAT |     1.8061 |        0 |    1.7893 |   0.0251 |
|      Clock [MHz] STAT     | 64397.0148 | 993.3320 | 3351.2206 | 894.4030 |

In [7]:
! cse142 run "sudo likwid-perfctr -g ENERGY ./programmer/madd_change_freq 8192 3700" | grep STAT

Sleeping longer as likwid_sleep() called without prior initialization
Sleeping longer as likwid_sleep() called without prior initialization
|   INSTR_RETIRED_ANY STAT   |  FIXC0  | 10568847420 |   0 | 10508326497 | 1.467895e+08 |
| CPU_CLK_UNHALTED_CORE STAT |  FIXC1  |  4229643210 |   0 |  4185296947 | 5.874504e+07 |
|  CPU_CLK_UNHALTED_REF STAT |  FIXC2  |  2639936780 |   0 |  2610618772 | 3.666579e+07 |
|       TEMP_CORE STAT       |   TMP0  |        2184 |  24 |          39 |      30.3333 |
|     PWR_PKG_ENERGY STAT    |   PWR0  |    158.9066 |   0 |     88.0086 |       2.2070 |
|     PWR_PP0_ENERGY STAT    |   PWR1  |           0 |   0 |           0 |            0 |
|    PWR_DRAM_ENERGY STAT    |   PWR3  |     20.3373 |   0 |     12.6387 |       0.2825 |
|  Runtime (RDTSC) [s] STAT |   114.1776 |   1.5858 |    1.5858 |    1.5858 |
| Runtime unhalted [s] STAT |     1.8391 |        0 |    1.8197 |    0.0255 |
|      Clock [MHz] STAT     | 85943.0513 | 999.8241 | 3687.3044 | 1193.653